![cactus](http://cactuscode.org/global/images/cactuslogo.png)
# Tutorial 2: Reworking Symmetry Boundary Conditions

In [ ]:
import os
os.environ["PATH"]="/opt/conda/envs/python2/bin:"+os.environ["PATH"]
!python --version

In [ ]:
%cd ~/
%mkdir Tutorial
%cd ~/Tutorial
!curl -kLO https://raw.githubusercontent.com/gridaphobe/CRL/ET_2018_02/GetComponents
!chmod a+x ./GetComponents
!echo no|./GetComponents --parallel ~/PreSyncTutorial/thorns.th
%cd ~/Tutorial/CactusPre
!./simfactory/bin/sim setup-silent
%cd ~/Tutorial/CactusPre
!time ./simfactory/bin/sim build -j 2 --thornlist=~/Tutorial/CactusPre/thornlists/thorns.th

<h1>Running Cactus!</h1>

In the first tutorial, we successfully updated a thorn to use PreSync. This demonstrated two different update types:
1. updating a thorn that doesn't include boundary routines
    1. add READ/WRITE declarations to scheduled functions
    2. place all old boundary scheduling into a legacy if statement
    3. add scheduling of BC selection routines in the PreSync_Selection group
    4. change DECLARE_CCTK_ARGUMENTS to the new macro and include the cctk_Arguments_Checked.h header file
2. Updating a boundary condition registered and applied using the Boundary thorn
    1. apply the changes for a standard thorn
    2. include the PreSync.h header file in the files with the boundary condition and registration functions

The second case has an important qualifier: it was specifically for boundary conditions which use Boundary to apply their boundary conditions. How do we change a boundary routine which doesn't use Boundary? In addition to this, there exists another less obvious case to consider. Symmetry boundary conditions (scheduled in BoundaryConditions group in Boundary) need to be applied to all variables, unlike physical boundary conditions. This means that PreSync needs to use a different system to apply them than the physical boundary conditions.

in this tutorial, we examine the symmetry boundary condition registered by the thorn CartGrid3D thorn and how this thorn is changed to use PreSync. To start, we need to access CartGrid3D without the PreSync changes. To do this, we will copy the master branch into the Tutorial directory so we may compare more easily inside this notebook.

In [ ]:
%cd ~/Tutorial/CactusPre/repos/cactusbase
!git checkout master
%cp -r CartGrid3D ~/Tutorial/
!git checkout presync

Below, we use diff to see how the schedule.ccl changed for PreSync. The first difference is simply a renamed function, so we can ignore that change. Several READ/WRITE declarations follow, as we would expect. The last change is different. Originally, symmetry boundary conditions were applied by the Boundary thorn. Symmetry functions were registered with Symbase, and then the ApplyBC function was scheduled inside te BoundaryConditions group provided by the Boundary thorn. This scheduled function should only run if PreSync is not active, so it checks the value of use_psync. In addition, the name changes from CartGrid3D_ApplyBC to Old_CartGrid3D_ApplyBC. This is because the argument lists are different for the two functions, so they must be declared separately. We will see the details of this change later.

In [ ]:
!diff ~/Tutorial/CactusPre/repos/cactusbase/CartGrid3D/schedule.ccl ~/Tutorial/CartGrid3D/schedule.ccl

Next, we need to look at the interface.ccl. Below, we can see that a new function has been added named Boundary_RegisterSymmetryBC. This is a new function provided by Boundary2 to register symmetry boundary conditions with PreSync. The implementation is very similar to the Boundary_RegisterPhysicalBC provided by Boundary and Boundary2. Any thorn which provides symmetry boundary conditions must use this function to properly interface with PreSync.

In [ ]:
!diff ~/Tutorial/CactusPre/repos/cactusbase/CartGrid3D/interface.ccl ~/Tutorial/CartGrid3D/interface.ccl

We can now move to the source code. Several files use the DECLARE_CCTK_ARGUMENTS macro, so these are changed to the new macro. However, this is a simple change, so we won't discuss it further. The next file with meaningful changes is RegisterSymmetries.c, which contains the function that registers the symmetry boundary condition with SymBase. The first difference is just a new name for the function, which we saw earlier in the schedule.ccl. the second change swaps out the macros. last change adds several new lines of code. We see that it is only activated if PreSync is turned on, in which case it calls the Boundary_RegisterSymmetryBC function. We could also have added a completely new function to register with PreSync, but we chose to just have a single registration function.

In [ ]:
!diff ~/Tutorial/CactusPre/repos/cactusbase/CartGrid3D/src/RegisterSymmetries.c ~/Tutorial/CartGrid3D/src/RegisterSymmetries.c

We only have two more files with changes! The first is the header file Symmetry.h. For PreSync, we included PreSync.h and also included a prototype for the ApplyBC function. The prototyping is so we can pass that function in the Boundary_RegisterSymmetryBC call.

In [ ]:
!diff ~/Tutorial/CactusPre/repos/cactusbase/CartGrid3D/src/Symmetry.h ~/Tutorial/CartGrid3D/src/Symmetry.h

The last file has the most significant changes, as it contains the symmetry function CartGrid3D_ApplyBC. Below, we extract 

In [ ]:
!sed -n 680,741p ~/Tutorial/CartGrid3D/src/Symmetry.c > ~/Tutorial/oldsym.c
!sed -n 680,700p ~/Tutorial/CactusPre/repos/cactusbase/CartGrid3D/src/Symmetry.c > ~/Tutorial/newsym.c
!sed -n 1,3p ~/Tutorial/oldsym.c
!sed -n 1,4p ~/Tutorial/newsym.c

In [ ]:
!sed -n 680,741p ~/Tutorial/CartGrid3D/src/Symmetry.c

In [ ]:
!sed -n 680,700p ~/Tutorial/CactusPre/repos/cactusbase/CartGrid3D/src/Symmetry.c

L

Changes to CartGrid3D:
    5. Add PreSync-enabled registration in SymbaseRegister function
    6. Add PreSync.h to Symmetry.h
    7. Create new ApplyBCs function
    8. Copy old function into Old_*

Data can be found in this directory. Using the next couple of commands, we will browse it.

In [ ]:
%cd ~/simulations/oldwave/output-0000/oldwave

In [ ]:
%ls *.asc

In [ ]:
# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
# https://matplotlib.org/examples/color/colormaps_reference.html
cmap = cm.gist_rainbow

The top of %pycat command showed us what the columns mean:
HYDROBASE::rho (hydrobase-rho)
* 1 iteration
* 2 time - how much time has passed in the simulation
* 3 the data, in this case the variable rho

Once we know all this, it is straightforward to plot the data. Python knows how to read regularly formatted text files that use the # character for comments. Fortunately, that's what Cactus produces in its asc files.

In [ ]:
file_data = np.genfromtxt("psi.xy.asc")

In [ ]:
sets = np.unique(file_data[:,0])
width = 8
height = 4
print("sets=",sets)
mn, mx = np.min(file_data[:,8]),np.max(file_data[:,8])
for which in sets: 
    print("which=",which)
    g = file_data[file_data[:,0]==which,:]
    x = g[:,5]
    y = g[:,6]
    z = g[:,8]
    zi = z.reshape(len(np.unique(y)),len(np.unique(x)))
    print('min/max=',np.min(zi),np.max(zi))
    plt.figure(figsize=(width, height))
    plt.imshow(zi[::-1,:],cmap,clim=(mn,mx))
    plt.show()

<h2>Using PreSync</h2>
We have confirmed that OldWave works, and by extension that legacy code still functions with the PreSync update to Cactus. Our next task is to change OldWave to use PreSync and remain backwards compatible with code that has not been updated to use PreSync.

To begin, we will change the schedule.ccl to use PreSync while remaining backward compatible. First, let us examine the current schedule.

In [ ]:
%pycat ~/Tutorial/CactusPre/repos/PresyncWave/OldWave/schedule.ccl

Currently, scheduled functions have SYNC statements to trigger synchronization, and the Boundary Routines section handles the registration and application of boundary conditions. PreSync instead relies on read/write declarations to determine when synchronization and boundary condition application routines should run. PreSync ignores the SYNC statements, so we can safely leave them for backward compatibility and add the READS and WRITES statements.

Read/write statements require the thorn::variable that is accessed and also the region on which it is accessed. PreSync supports several regions of validity, but for most functions the only two which should be used are "interior" and "everywhere". A variable which is read "everywhere" requires that the variable be valid in the ghost zones and boundary regions, while reading the interior does not. Writing variables changes their region of validity, causing synchronization to trigger when the next read(everywhere) is encountered.

As an example, let us look at the function presync_wave_init. It is scheduled as

```
schedule oldsync_wave_init at CCTK_POSTINITIAL
{
  LANG: C
  SYNC: OldWave::evo_vars
} "initial condition"
```

This simple function writes psi and phi initial data, and it reads the grid variables x, and y. The code only loops over the interior, so Therefore, we may change the scheduling to the following:

```
schedule oldsync_wave_init at CCTK_POSTINITIAL
{
  LANG: C
  READS: Grid::x(interior), y
  WRITES: OldWave::evo_vars(interior)
  SYNC: OldWave::evo_vars
} "initial condition"
```

As seen in the READ declaration, multiple variables can be listed on one line by adding them after the region specification. Also, a group may be given instead of individual variables, as in the WRITE declaration.

Compared to the confusing and non-trivial problem of determining the correct positions of the SYNC statements, determining the read/write declarations for a function is far simpler and only requires knowledge of the one scheduled function (and any functions it calls).

All the functions in OldWave have very simple read/write dependencies, and adding those to the schedule.ccl are all that is required to properly trigger synchronization with PreSync. To see the complete declarations, you may look further down where we write the new schedule.ccl.

For now, we move on to boundary conditions. This thorn uses its own boundary conditions, which it registers with the Boundary thorn in the presync_registervars function. To apply boundary conditions, it schedules a SelectBCs function and the ApplyBCs group (provided by Boundary) wherever boundary conditions should be applied. In this case, there are two separate SelectBCs functions, and they must be scheduled separately.

In contrast, PreSync only requires the SelectBCs routine to run once, at the beginning of a simulation. This is facillitated by the Boundary2 thorn, which provides the features of Boundary updated to use PreSync. In addition, it provides groups for boundary registration and selection. The new schedule for the boundary routines is
```
schedule presync_registerboundary in PreSync_Registration
{
  LANG: C
} "register boundaries"

schedule presync_SelectBCs in PreSync_Selection
{
  LANG: C
} "select boundary conditions"

schedule energy_SelectBCs in PreSync_Selection
{
  LANG: C
} "select boundary conditions"
```

While the selectBCs functions could be combined, they remain separate to provide backward compatibility. However, we can't simply add this to the schedule.ccl and expect it to work. We need to turn off the new scheduled routines and turn on the old scheduled routines if running in backward-compatible mode. This is accomplished by using the routine
```
CCTK_ParameterValInt("use_psync","Carpet")
```
to determine which should run. If it equals 1, then PreSync is active. If it equals 0, then the backward-compatible routines should be used.

The changes discussed above are implemented in the schedule.ccl below. This command will overwrite the old schedule.ccl with our new version.

In [ ]:
%%writefile ~/Tutorial/CactusPre/repos/PresyncWave/OldWave/schedule.ccl

# Schedule definitions for thorn PresyncWave
storage: rhs_vars[3], evo_vars[3], evo_divs, wave_energy[3]

schedule oldsync_wave_init at CCTK_POSTINITIAL
{
  LANG: C
  READS: Grid::x(interior), y
  WRITES: OldWave::evo_vars(interior)
  SYNC: OldWave::evo_vars
} "initial condition"

schedule oldsync_wave_evolve in MoL_CalcRHS
{
  LANG: C
  READS: OldWave::evo_divs(interior), phi
  WRITES: OldWave::rhs_vars(interior)
  SYNC: OldWave::rhs_vars
} "Evolve loop"

schedule oldsync_derivatives in MoL_CalcRHS before oldsync_wave_evolve
{
  LANG: C
  READS: OldWave::evo_vars(everywhere)
  WRITES: OldWave::evo_divs(interior)
} "Compute derivatives"

schedule oldsync_registervars in MoL_Register
{
  LANG: C
  OPTIONS: META
}"Register funwave variables for MoL"

#################################################
##             Boundary Routines               ##
#################################################

# In this function, we register boundary conditions
# with Carpet (not thorn Boundary). This new way
# of doing things makes it possible for Carpet to
# fill in the exterior of the variable at the
# same time it performs a sync.
if(CCTK_ParameterValInt("use_psync","Carpet") == 1) {
  schedule oldsync_registerboundary in PreSync_Registration
  {
    LANG: C
  } "register boundaries"

  schedule oldsync_evolve_SelectBCs in PreSync_Selection
  {
    LANG: C
  } "select boundary conditions"

  schedule oldsync_energy_SelectBCs in PreSync_Selection
  {
    LANG: C
  } "select boundary conditions"
}

if(CCTK_ParameterValInt("use_psync","Carpet") == 0) {
  schedule group PSWave_Boundaries in MoL_CalcRHS before oldsync_derivatives
  {
  } "boundary condition group"
  schedule group PSWave_Boundaries in CCTK_ANALYSIS before oldsync_energy
  {
  } "boundary condition group"

  schedule group PSWave_Boundaries at POSTRESTRICT
  {
  } "boundary condition group"

  schedule oldsync_registerboundary at CCTK_WRAGH
  {
    LANG: C
  } "register boundaries"

  schedule oldsync_evolve_SelectBCs in PSWave_Boundaries
  {
    LANG: C
  } "select boundary conditions"

  schedule GROUP ApplyBCs as PSWave_ApplyBCs in PSWave_Boundaries after oldsync_evolve_selectBCs
  {
  } "Apply boundary conditions"

  schedule group Energy_Boundary at CCTK_ANALYSIS after oldsync_energy
  {
  } "boundary condition group"

  schedule oldsync_energy_SelectBCs in Energy_Boundary
  {
    LANG: C
  } "select boundary conditions"

  schedule GROUP ApplyBCs as Energy_ApplyBCs in Energy_Boundary after oldsync_energy_SelectBCs
  {
  } "Apply boundary conditions"
}

#################################################
##            Energy Calculations              ##
#################################################

schedule oldsync_energy at CCTK_ANALYSIS
{
  LANG: C
  READS: OldWave::evo_vars(everywhere)
  WRITES: OldWave::energy(interior)
  SYNC: OldWave::wave_energy
} "Calculate energy"

We now have a working schedule.ccl. None of the other ccl files need to be changed, so we can move on to the source code. First, we need to look at the PSWave.h. This provides the function type used to pass the boundary routines to the Boundary thorn. To do this, we could have used the Boundary.h thorn from Boundary or included the typedef itself. We elected to perform the latter, as we can see below.

In [ ]:
%cat ~/Tutorial/CactusPre/repos/PresyncWave/OldWave/src/PSWave.h

For PreSync, we replace this with the header file PreSync.h. This header file is located in the flesh, so it is readily available to all thorns. It contains the boundary_function typedef as well as the definitions for the regions of validity. In addition to variable registration, this is included whenever a function needs to access the validity of a variable.

In [ ]:
%%writefile ~/Tutorial/CactusPre/repos/PresyncWave/OldWave/src/PSWave.h

#ifndef _PSWave_H_
#define _PSWave_H_

#include "PreSync.h"

#ifdef __cplusplus
extern "C" {
#endif

/* prototype for routine registered as providing 'zero' boundary condition */
CCTK_INT fun_stwave(const cGH *cctkGH, CCTK_INT num_vars, CCTK_INT *var_indices,
                  CCTK_INT *faces, CCTK_INT *widths, CCTK_INT *table_handles);

/* prototype for routine registered as providing 'symmetry' and 'anti-symmetry' boundary conditions */
CCTK_INT fun_bf2(const cGH *cctkGH, CCTK_INT num_vars, CCTK_INT *var_indices,
                  CCTK_INT *faces, CCTK_INT *widths, CCTK_INT *table_handles);

#ifdef __cplusplus
}
#endif

#endif /* _PSWave_H_ */

The next step is very easy, though it will be disproportionately long in the notebook. The current method for accessing variables in Cactus is to use the DECLARE_CCTK_ARGUMENTS macro. However, this declares *every* variable and not just those needed by the function. To reduce these declarations and provide additional error-checking, PreSync provides new (optional) macros which are generated based on the read/write declarations. Each scheduled function has its own macro which only provides access to declared variables. In addition, read-only declarations are declared as const, which provides additional verification for read/write declarations. To include these new macros, we add
```
#include "cctk_Arguments_Checked.h"
```
and replace DECLARE_CCTK_ARGUMENTS with DECLARE_CCTK_ARGUMENTS_FunctionName. This is done below for all the code in the OldWave thorn.

In [ ]:
%%writefile ~/Tutorial/CactusPre/repos/PresyncWave/OldWave/src/energy.cc

#include "cctk.h" 
#include "cctk_Arguments.h"
#include "cctk_Arguments_Checked.h"
#include "cctk_Parameters.h"
#include "iostream"

void oldsync_energy(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS_oldsync_energy
  DECLARE_CCTK_PARAMETERS;
      
  const int imin0=cctk_nghostzones[0];
  const int imin1=cctk_nghostzones[1];
  const int imin2=cctk_nghostzones[2];
  const int imax0=cctk_lsh[0] - cctk_nghostzones[0];
  const int imax1=cctk_lsh[1] - cctk_nghostzones[1];
  const int imax2=cctk_lsh[2] - cctk_nghostzones[2];
  const int zero = CCTK_GFINDEX3D(cctkGH,0,0,0);
  const int di = (cctk_lsh[0]==1) ? 0:CCTK_GFINDEX3D(cctkGH,1,0,0) - zero;
  const int dj = (cctk_lsh[1]==1) ? 0:CCTK_GFINDEX3D(cctkGH,0,1,0) - zero;
  const int dk = (cctk_lsh[2]==1) ? 0:CCTK_GFINDEX3D(cctkGH,0,0,1) - zero;
  #pragma omp parallel
  CCTK_LOOP3(calc_energy,i,j,k,
    imin0,imin1,imin2,imax0,imax1,imax2,
    cctk_ash[0],cctk_ash[1],cctk_ash[2])
  {
    int cc = CCTK_GFINDEX3D(cctkGH,i,j,k);
    double psix = (psi[cc+di]-psi[cc-di])/(2.0*CCTK_DELTA_SPACE(0));
    double psiy = (psi[cc+dj]-psi[cc-dj])/(2.0*CCTK_DELTA_SPACE(1));
    double psiz = (psi[cc+dk]-psi[cc-dk])/(2.0*CCTK_DELTA_SPACE(2));
    energy[cc] = phi[cc]*phi[cc] + psix*psix +
      psiy*psiy + psiz*psiz;
  }
  CCTK_ENDLOOP3(calc_energy);
}

In [ ]:
%%writefile ~/Tutorial/CactusPre/repos/PresyncWave/OldWave/src/Wave.cc

#include <cctk.h>
#include <cctk_Arguments.h>
#include <cctk_Arguments_Checked.h>
#include <cctk_Parameters.h>
#include <iostream>

#define sq(X) (X)*(X)

extern "C"
void oldsync_wave_init(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS_oldsync_wave_init
  DECLARE_CCTK_PARAMETERS

  const int imin0=cctk_nghostzones[0];
  const int imin1=cctk_nghostzones[1];
  const int imin2=cctk_nghostzones[2];
  const int imax0=cctk_lsh[0] - cctk_nghostzones[0];
  const int imax1=cctk_lsh[1] - cctk_nghostzones[1];
  const int imax2=cctk_lsh[2] - cctk_nghostzones[2];
  CCTK_REAL x0 = x[CCTK_GFINDEX3D(cctkGH,cctk_lsh[0]/2,cctk_lsh[1]/2,cctk_lsh[2]/2)];
  CCTK_REAL y0 = x[CCTK_GFINDEX3D(cctkGH,cctk_lsh[0]/2,cctk_lsh[1]/2,cctk_lsh[2]/2)];
  #pragma omp parallel
  CCTK_LOOP3(calc_oldsync_wave_init,
    i,j,k, imin0,imin1,imin2, imax0,imax1,imax2,
    cctk_ash[0],cctk_ash[1],cctk_ash[2])
  {
    int cc = CCTK_GFINDEX3D(cctkGH,i,j,k);
    psi[cc] = exp(-sq(x[cc]-x0)-sq(y[cc]-y0));
    phi[cc] = 0;
  }
  CCTK_ENDLOOP3(calc_oldsync_wave_init);
}

extern "C"
void oldsync_wave_evolve(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS_oldsync_wave_evolve;
  DECLARE_CCTK_PARAMETERS;

  const int imin0=cctk_nghostzones[0];
  const int imin1=cctk_nghostzones[1];
  const int imin2=cctk_nghostzones[2];
  const int imax0=cctk_lsh[0] - cctk_nghostzones[0];
  const int imax1=cctk_lsh[1] - cctk_nghostzones[1];
  const int imax2=cctk_lsh[2] - cctk_nghostzones[2];
  #pragma omp parallel
  CCTK_LOOP3(calc_oldsync_wave_evol,
    i,j,k, imin0,imin1,imin2, imax0,imax1,imax2,
    cctk_ash[0],cctk_ash[1],cctk_ash[2])
  {
    int cc = CCTK_GFINDEX3D(cctkGH,i,j,k);
    psi_rhs[cc] = phi[cc];
    phi_rhs[cc] = dxx_psi[cc]+dyy_psi[cc]+dzz_psi[cc];
  }
  CCTK_ENDLOOP3(calc_oldsync_wave_evol);
}

extern "C"
void oldsync_derivatives(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS_oldsync_derivatives;
  DECLARE_CCTK_PARAMETERS;

  const int imin0=cctk_nghostzones[0];
  const int imin1=cctk_nghostzones[1];
  const int imin2=cctk_nghostzones[2];
  const int imax0=cctk_lsh[0] - cctk_nghostzones[0];
  const int imax1=cctk_lsh[1] - cctk_nghostzones[1];
  const int imax2=cctk_lsh[2] - cctk_nghostzones[2];
  const int zero = CCTK_GFINDEX3D(cctkGH,0,0,0);
  const int di = (cctk_lsh[0]==1) ? 0:CCTK_GFINDEX3D(cctkGH,1,0,0) - zero;
  const int dj = (cctk_lsh[1]==1) ? 0:CCTK_GFINDEX3D(cctkGH,0,1,0) - zero;
  const int dk = (cctk_lsh[2]==1) ? 0:CCTK_GFINDEX3D(cctkGH,0,0,1) - zero;
  assert(!std::isnan(psi[zero]));
  #pragma omp parallel
  CCTK_LOOP3(calc_oldsync_derivs,
    i,j,k, imin0,imin1,imin2, imax0,imax1,imax2,
    cctk_ash[0],cctk_ash[1],cctk_ash[2])
  {
    int cc = CCTK_GFINDEX3D(cctkGH,i,j,k);
    dxx_psi[cc] = (psi[cc+di]+psi[cc-di]-2.0*psi[cc])/(CCTK_DELTA_SPACE(0)*CCTK_DELTA_SPACE(0));
    dyy_psi[cc] = (psi[cc+dj]+psi[cc-dj]-2.0*psi[cc])/(CCTK_DELTA_SPACE(1)*CCTK_DELTA_SPACE(1));
    dzz_psi[cc] = (psi[cc+dk]+psi[cc-dk]-2.0*psi[cc])/(CCTK_DELTA_SPACE(2)*CCTK_DELTA_SPACE(2));
  }
  CCTK_ENDLOOP3(calc_oldsync_derivs);
}

Congratulations! You've just changed your first thorn to use PreSync! To verify that we haven't broken backward compatibility, we can recompile and run the test in OldWave. The parameter file for this test is identical to the one we used earlier, except it calculates less iterations.

In [ ]:
%cd ~/Tutorial/CactusPre/
!time ./simfactory/bin/sim build -j 2 --thornlist=~/Tutorial/CactusPre/thornlists/thorns.th

In [ ]:
%cd ~/Tutorial/CactusPre/
!./simfactory/bin/sim create-run mytest --testsuite --procs 2 --num-threads 1 --select-tests PresyncWave/OldWave

<h3>Questions and Exercises:</h3>

* Run the above simulation using a single process instead of two. Do the plotting routines work? What changes did you have to make. What would you need to do to make it work with 3?
* Run the code at 1/2 the resolution.
* Position the Guassian wave at a different place on the grid.
* If you wanted to change the compiler or a compiler flag, how would you go about doing that?
* If you wanted to add another thorn to the list of thorns to compile, how would you go about doing that?
* If you wanted to create a thornlist that would check out Cactus under the Foo directory instead of the CactusFW2 directory, how would you do it?

<table><tr><td>This work sponsored by NSF grants <a href="https://www.nsf.gov/awardsearch/showAward?AWD_ID=1550551"> OAC 1550551</a> and <a href="https://www.nsf.gov/awardsearch/showAward?AWD_ID=1539567"> CCF 1539567</a></td><td><img src="https://www.nsf.gov/awardsearch/images/common/nsf_logo_bottom.png"></tr></table>